In [1]:
!pip install emoji

You should consider upgrading via the 'c:\python\python38\python.exe -m pip install --upgrade pip' command.



  Created wheel for emoji: filename=emoji-0.5.4-py3-none-any.whl size=42179 sha256=44795b90efe0ee81a2b4805c6c36f2b89d5d0f2e271efb919eab4ca350f0c458
  Stored in directory: c:\users\91768\appdata\local\pip\cache\wheels\22\d2\67\010d63edb25b9f0e7d5c181a4b77d9bdc9e027eb5562c4cc1d
Successfully built emoji


In [2]:
import emoji

In [4]:
#emoji.EMOJI_UNICODE

In [5]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                    "5": ":hundred_points:",
                    "6": ":fire:",
                    "7": ":face_blowing_a_kiss:",
                    "8": ":chestnut:",
                    "9": ":flexed_biceps:"
                   }

In [6]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴
💯
🔥
😘
🌰
💪


In [67]:
import pandas as pd

train = pd.read_csv("D:/CB Git/machine-learning-online-2018/20. RNN/Emoji Predictor/dataset/train_emoji.csv", header=None)
test = pd.read_csv("D:/CB Git/machine-learning-online-2018/20. RNN/Emoji Predictor/dataset/test_emoji.csv", header=None)

In [68]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [69]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

In [70]:
# printing sentences along with emojis
for i in range(10):
    print(X_train[i],emoji.emojize(emoji_dictionary[str(Y_train[i])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


## __Converting Sentences into Embeddings__

In [71]:
f = open("./Transfer Learning using NLP/glove.6B.50d.txt", encoding='utf8')

In [72]:
import numpy as np
embedding_index = {}

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float')
    embedding_index[word] = coefs
    
f.close()

In [73]:
print(len(embedding_index))

400000


## **Converting Sentence into vectors**

In [74]:
emd_dim = embedding_index['eat'].shape[0]
print(emd_dim)

50


In [75]:
def embeddingOut(X):
    maxlen = 10
    embedding_out = np.zeros((X.shape[0],maxlen,emd_dim))
    
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        
        for jx in range(len(X[ix])):
            try:
                embedding_out[ix][jx] = embedding_index[X[ix][jx].lower()]
            except:
                embedding_out[ix][jx] = np.zeros((50,))
    return embedding_out

In [52]:
#for i in range(X_train.shape[0]):
    #X_train[i] = str(X_train[i])

<ipython-input-52-1ff9df3ea8f3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[i] = str(X_train[i])


In [76]:
emb_X_train = embeddingOut(X_train)
emb_X_test = embeddingOut(X_test)

<ipython-input-75-f782d6e218ac>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[ix] = X[ix].split()


In [77]:
print(X_train[0])

['never', 'talk', 'to', 'me', 'again']


In [78]:
print(emb_X_train.shape)
print(emb_X_test.shape)

(132, 10, 50)
(56, 10, 50)


In [79]:
print(Y_train.shape)

(132,)


In [80]:
from keras.utils import np_utils

Y_train = np_utils.to_categorical(Y_train, num_classes=5)
Y_test = np_utils.to_categorical(Y_test, num_classes=5)

In [81]:
print(Y_train.shape)

(132, 5)


## **LSTM Implementation**

In [64]:
from keras.layers import *
from keras.models import Sequential

In [92]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer ='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [98]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpoint = ModelCheckpoint("best_model.h5",monitor='val_loss',verbose=True,save_best_only=True)
earlystop = EarlyStopping(monitor='val_acc',patience=10)

hist = model.fit(emb_X_train,Y_train,epochs=150,batch_size=64,shuffle=True,validation_split=0.2)

Train on 105 samples, validate on 27 samples
Epoch 1/150
105/105 [==============================] - 0s 427us/step - loss: 0.0144 - accuracy: 1.0000 - val_loss: 1.9378 - val_accuracy: 0.6667
Epoch 2/150
105/105 [==============================] - 0s 418us/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 1.9522 - val_accuracy: 0.6667
Epoch 3/150
105/105 [==============================] - 0s 408us/step - loss: 0.0042 - accuracy: 1.0000 - val_loss: 1.9632 - val_accuracy: 0.6667
Epoch 4/150
105/105 [==============================] - 0s 437us/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 1.9698 - val_accuracy: 0.6296
Epoch 5/150
105/105 [==============================] - 0s 418us/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 1.9766 - val_accuracy: 0.6296
Epoch 6/150
105/105 [==============================] - 0s 494us/step - loss: 0.0104 - accuracy: 1.0000 - val_loss: 2.0169 - val_accuracy: 0.6667
Epoch 7/150
105/105 [==============================] - 0s 418us/step - loss: 0.0057 -

Epoch 57/150
105/105 [==============================] - 0s 389us/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 2.2276 - val_accuracy: 0.6667
Epoch 58/150
105/105 [==============================] - 0s 446us/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 2.2303 - val_accuracy: 0.6667
Epoch 59/150
105/105 [==============================] - 0s 437us/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 2.2344 - val_accuracy: 0.6667
Epoch 60/150
105/105 [==============================] - 0s 399us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 2.2419 - val_accuracy: 0.6667
Epoch 61/150
105/105 [==============================] - 0s 456us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 2.2504 - val_accuracy: 0.6667
Epoch 62/150
105/105 [==============================] - 0s 418us/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 2.2629 - val_accuracy: 0.6667
Epoch 63/150
105/105 [==============================] - 0s 437us/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 2.2756 -

105/105 [==============================] - 0s 456us/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 2.2965 - val_accuracy: 0.6296
Epoch 114/150
105/105 [==============================] - 0s 446us/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 2.3072 - val_accuracy: 0.6296
Epoch 115/150
105/105 [==============================] - 0s 408us/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 2.3175 - val_accuracy: 0.6296
Epoch 116/150
105/105 [==============================] - 0s 399us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 2.3278 - val_accuracy: 0.6296
Epoch 117/150
105/105 [==============================] - 0s 418us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 2.3371 - val_accuracy: 0.6296
Epoch 118/150
105/105 [==============================] - 0s 437us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 2.3446 - val_accuracy: 0.6296
Epoch 119/150
105/105 [==============================] - 0s 456us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 2.3512 - val_ac

In [99]:
pred = model.predict_classes(emb_X_test)

In [100]:
print(pred)

[4 3 3 2 2 2 3 2 4 2 1 2 0 3 1 3 2 2 3 2 0 0 4 0 3 3 2 0 0 2 0 1 3 2 0 2 2
 4 4 2 1 0 0 1 2 2 2 2 3 1 3 0 3 2 2 3]


In [104]:
#model.load_weights("best_model.h5")

In [102]:
model.evaluate(emb_X_test,Y_test)

56/56 [==============================] - 0s 214us/step


[2.854450617517744, 0.625]

In [103]:
for i in range(30):
    print(' '.join(X_test[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Y_test[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😓
😓
he got a raise
😁
😓
she got me a present
❤️
😁
ha ha ha it was so funny
😁
😁
he is a good friend
❤️
😁
I am upset
❤️
😓
We had such a lovely dinner tonight
❤️
😁
where is the food
🍴
🍴
Stop making this joke ha ha ha
😁
😁
where is the ball
⚾
⚾
work is hard
😓
😁
This girl is messing with me
😓
❤️
are you serious ha ha
😁
😓
Let us go play baseball
⚾
⚾
This stupid grader is not working
😓
😓
work is horrible
😓
😁
Congratulation for having a baby
😁
😁
stop messing around
😓
😓
any suggestions for dinner
🍴
😁
I love taking breaks
❤️
❤️
you brighten my day
😁
❤️
I boiled rice
🍴
🍴
she is a bully
😓
❤️
Why are you feeling bad
😓
😓
I am upset
😓
😓
I worked during my birthday
😓
😁
My grandmother is the love of my life
❤️
❤️
enjoy your break
😁
❤️
valentine day is near
❤️
😁
